<a href="https://colab.research.google.com/github/Waleed-Daud/AS/blob/master/Representation_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow.contrib import eager
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [0]:
! git clone  https://github.com/Waleed-Daud/AS.git
% cd AS 

In [0]:
# MMD special imports

from functools import partial
import utils
slim = tf.contrib.slim

## Q1. Let’s start by just looking at the marginal distribution of each feature in each group (A = 0, A = 1). For each feature, fit a Gaussian to that feature for each group – this should give us two Gaussians (parameters μ 0 , σ 0 or μ 1 , σ 1 ) for each feature. Then, we’ll use these simple distributions to preprocess the features of group A = 0 and 1 so they more closely match each other. For each feature x for a point in group A = a, let the pre-processed a feature x 0 = x−μ σ a . This pre-processing step should match the first two moments of the features of each group. As in the previous question, learn a classifier g to predict Y and a classifier h to predict A from this pre-processed dataset. Report the accuracy and ∆ DP for g and accuracy and reweighted accuracy for h. What happened?

In [0]:
tf.enable_eager_execution()

In [0]:
def filter_dataframe(dataset,key,label):
  
  female_data_binary = dataset[key] == 1
  male_data_binary = dataset[key] == 0
  
  male_data = dataset[male_data_binary]
  female_data = dataset[female_data_binary]
  
  sex_male_label = male_data[label]
  sex_female_label = female_data[label]
  
  sex_male_data  = male_data.drop(['y','A'], axis =1)
  sex_female_data  = female_data.drop(['y','A'], axis =1)
  
  
  return sex_male_data,sex_male_label, sex_female_data,sex_female_label




def filter_dataframe_dp(dataset,key,label):
  
  if key =='sex_Female':
    female_data_binary = dataset[key] == 1
    male_data_binary = dataset[key] == 0
  
    male_data = dataset[male_data_binary]
    female_data = dataset[female_data_binary]
    
  else:
    
    female_data_binary = dataset['A'] == 1
    male_data_binary = dataset['A'] == 0
  
    male_data = dataset[male_data_binary]
    female_data = dataset[female_data_binary]
  
  
  sex_male_label = male_data[label]
  sex_female_label = female_data[label]
  
  
  A_male = male_data['A']
  A_female = female_data['A']
  
  
  sex_male_data  = male_data.drop(['y','A'], axis =1)
  sex_female_data  = female_data.drop(['y','A'], axis =1)
  
  
  return sex_male_data,sex_male_label,A_male  , sex_female_data,sex_female_label,A_female




def convert_to_tensors(mlist):
  tlist = []
  for tensor in mlist:
    tlist.append(tf.convert_to_tensor(tensor,dtype=tf.float32))
  return tlist


class Custom_Loss:
  
  
  def __init__(self):
    pass
  

  def maximum_mean_discrepancy(self,x, y, kernel=utils.gaussian_kernel_matrix):
    r"""Computes the Maximum Mean Discrepancy (MMD) of two samples: x and y.
    Maximum Mean Discrepancy (MMD) is a distance-measure between the samples of
    the distributions of x and y. Here we use the kernel two sample estimate
    using the empirical mean of the two distributions.
    MMD^2(P, Q) = || \E{\phi(x)} - \E{\phi(y)} ||^2
                = \E{ K(x, x) } + \E{ K(y, y) } - 2 \E{ K(x, y) },
    where K = <\phi(x), \phi(y)>,
      is the desired kernel function, in this case a radial basis kernel.
    Args:
        x: a tensor of shape [num_samples, num_features]
        y: a tensor of shape [num_samples, num_features]
        kernel: a function which computes the kernel in MMD. Defaults to the
                GaussianKernelMatrix.
    Returns:
        a scalar denoting the squared maximum mean discrepancy loss.
    """
    with tf.name_scope('MaximumMeanDiscrepancy'):
      # \E{ K(x, x) } + \E{ K(y, y) } - 2 \E{ K(x, y) }
      cost = 0
      for i in range(x.shape[0]):
        
        cost += tf.reduce_mean(kernel(x[i:i+64], x[i:i+64]))         # Batch Size = 64
      
      for i in range(y.shape[0]):
        cost += tf.reduce_mean(kernel(y[i:i+64], y[i:i+64]))         # Batch Size = 64
      
      for i in range(x.shape[0]):
        cost -= 2 * tf.reduce_mean(kernel(x[i:i+64], y[i:i+64]))      # Batch Size = 64

      # We do not allow the loss to become negative.
      cost = tf.where(cost > 0, cost, 0, name='value')
      return cost

    
    
  def mmd_loss(self, source_samples, target_samples, weight, scope=None):
    """Adds a similarity loss term, the MMD between two representations.
    This Maximum Mean Discrepancy (MMD) loss is calculated with a number of
    different Gaussian kernels.
    Args:
      source_samples: a tensor of shape [num_samples, num_features].
      target_samples: a tensor of shape [num_samples, num_features].
      weight: the weight of the MMD loss.
      scope: optional name scope for summary tags.
    Returns:
      a scalar tensor representing the MMD loss value.
    """
    sigmas = [
        1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 5, 10, 15, 20, 25, 30, 35, 100,
        1e3, 1e4, 1e5, 1e6
    ]
    gaussian_kernel = partial(
        utils.gaussian_kernel_matrix, sigmas=tf.constant(sigmas))

    loss_value = self.maximum_mean_discrepancy(
        source_samples, target_samples, kernel=gaussian_kernel)
    loss_value = tf.maximum(1e-4, loss_value) * weight
    assert_op = tf.Assert(tf.is_finite(loss_value), [loss_value])
    with tf.control_dependencies([assert_op]):
      tag = 'MMD Loss'
      if scope:
        tag = scope + tag
      tf.summary.scalar(tag, loss_value)
      tf.losses.add_loss(loss_value)

    return loss_value

In [0]:
# Reading Data

train_data = np.load("./adult/adult_train.npz")

test_data = np.load("./adult/adult_test.npz")


X_train = train_data.f.x
y_train = train_data.f.y
A_train = train_data.f.a

X_test = test_data.f.x
y_test = test_data.f.y
A_test = test_data.f.a

In [0]:
columns_file = open('./adult/adult_headers.txt','r')
columns = columns_file.read()
columns_names = columns.split("\n")

X_train_frame = pd.DataFrame(X_train,columns=columns_names) 
y_train_frame = pd.DataFrame(y_train,columns=['y'])
A_train_frame = pd.DataFrame(A_train,columns=['A'])

X_test_frame = pd.DataFrame(X_test,columns=columns_names) 
y_test_frame = pd.DataFrame(y_test,columns=['y']) 
A_test_frame = pd.DataFrame(A_test,columns=['A'])

dataset_train_frame = pd.concat([X_train_frame,y_train_frame,A_train_frame],axis=1)
dataset_train_frame.fillna(0,inplace = True)

dataset_test_frame =  pd.concat([X_test_frame,y_test_frame,A_test_frame],axis=1)
dataset_test_frame.fillna(0,inplace = True)


### 1- Normalize the Data.

In [0]:
X_train_a0, y_train_a0, X_train_a1, y_train_a1 = filter_dataframe(dataset_train_frame,'A','y') 
X_test_a0, y_test_a0, X_test_a1, y_test_a1  = filter_dataframe(dataset_test_frame,'A','y')


X_train_a0_norm = ( X_train_a0 - X_train_a0.mean() ) / X_train_a0.std()
X_train_a1_norm = ( X_train_a1 - X_train_a1.mean() ) / X_train_a1.std()

X_test_a0_norm = ( X_test_a0 - X_train_a0.mean() ) / X_train_a0.std()
X_test_a1_norm = ( X_test_a1 - X_train_a1.mean() ) / X_train_a1.std()






X_train_frame = pd.concat([X_train_a0_norm,X_train_a1_norm], axis=0)
X_test_frame =  pd.concat([X_test_a0_norm,X_test_a1_norm], axis=0)

y_train_frame = pd.concat([y_train_a0,y_train_a1], axis=0)
y_test_frame =  pd.concat([y_test_a0,y_test_a1], axis=0)


# Shuffling the training data

gen_nums = tf.cast(tf.linspace(0.0,X_train_frame.shape[0]-1,X_train_frame.shape[0]-1), dtype=tf.int32)
indexes = tf.random.shuffle(gen_nums)


X_train = X_train_frame.get_values()[indexes,:]    # numpy array
y_train = y_train_frame.get_values()[indexes]
A_train = A_train_frame.get_values()[indexes]


# Shuffling the testing data

gen_nums = tf.cast(tf.linspace(0.0,X_test.shape[0]-1,X_test.shape[0]-1), dtype=tf.int32)
indexes = tf.random.shuffle(gen_nums)


X_test = X_test_frame.get_values()[indexes,:]   # numpy array
y_test = y_test_frame.get_values()[indexes]
A_test = A_test_frame.get_values()[indexes]

###############################################################################
m,n = X_train.shape

m2, n2 = X_test.shape



# Convert to Tensors

X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
X_test = tf.convert_to_tensor(X_test, dtype=tf.float32)

y_train = tf.convert_to_tensor(y_train, dtype= tf.float32)
y_train = tf.reshape(y_train,(m,1))
y_test = tf.convert_to_tensor(y_test, dtype = tf.float32)
y_test = tf.reshape(y_test,(m2,1))



A_train = tf.convert_to_tensor(A_train, dtype = tf.float32)
A_train = tf.reshape(A_train,(m,1))

A_test = tf.convert_to_tensor(A_test, dtype = tf.float32)
A_test = tf.reshape(A_test,(m2,1))


In [10]:
loss = Custom_Loss()


loss.mmd_loss(X_train[:100000,:],X_test[:100000,:],0.1)


INFO:tensorflow:Summary name MMD Loss is illegal; using MMD_Loss instead.


<tf.Tensor: id=2995575, shape=(), dtype=float32, numpy=1e-05>

In [0]:
def Binary(logits):
  
  y_soft = tf.sigmoid(logits)
  y = tf.cast((y_soft>=0.5),dtype= tf.float32)
  
  return y

def accuracy(y,y_):
    y_ = tf.nn.sigmoid(y_)
    ans = tf.cast((y_>=0.5),dtype= tf.float32)
    res = tf.cast(tf.equal(ans,y),tf.float32)
    
    return tf.reduce_mean(res)
  

def RW_accuracy(y1,y1_, y2, y2_):
    rw = tf.multiply(tf.constant(0.5),(accuracy(y1,y1_) + accuracy(y2,y2_) ))
    return rw


def delta_dp(model, dataset,key,label):
  
  DP_male_data_frame,DP_male_label_frame,A_male_frame, DP_female_data_frame, DP_female_label_frame,A_female_frame = filter_dataframe_dp(dataset,key,label)
  
  
  
  DP_male_data = tf.convert_to_tensor(DP_male_data_frame.to_numpy() ,dtype=tf.float32) 
  DP_female_data = tf.convert_to_tensor(DP_female_data_frame.to_numpy() ,dtype=tf.float32) 
  
  DP_male_label = tf.convert_to_tensor(DP_male_label_frame.to_numpy() ,dtype=tf.float32) 
  DP_female_label = tf.convert_to_tensor(DP_female_label_frame.to_numpy() ,dtype=tf.float32) 
  
  A_DP_male = tf.convert_to_tensor(A_male_frame.to_numpy(), dtype= tf.float32)
  A_DP_female = tf.convert_to_tensor(A_female_frame.to_numpy(), dtype= tf.float32)

  DP_male_y_ = model(DP_male_data)
  DP_female_y_ = model(DP_female_data)

  
  
  DP_female = tf.reduce_mean(tf.multiply(DP_female_y_ , tf.subtract(tf.constant(1.0),A_DP_female)))
  DP_male = tf.reduce_mean(tf.multiply(DP_male_y_ , A_DP_male))
  
  delta_DP = tf.abs(DP_female - DP_male )
 

  print("Female Accuracy: {}".format(accuracy(DP_female_label,DP_female_y_) ))
  print("Male Accuracy: {}".format(accuracy(DP_male_label,DP_male_y_) ))
  
  return delta_DP.numpy()



def RW_accuracy_v2(model, dataset,key,label):
  
    male_data_frame,male_label_frame, female_data_frame, female_label_frame = filter_dataframe(dataset,key,label)
   
    male_data = tf.convert_to_tensor(male_data_frame.to_numpy() ,dtype=tf.float32) 
    female_data = tf.convert_to_tensor(female_data_frame.to_numpy() ,dtype=tf.float32) 
    
    male_label = tf.convert_to_tensor(male_label_frame.to_numpy() ,dtype=tf.float32) 
    female_label = tf.convert_to_tensor(female_label_frame.to_numpy() ,dtype=tf.float32) 
    
    male_y_ = model(male_data)
    female_y_ = model(female_data)
   

    rw = RW_accuracy(female_label,female_y_,male_label,male_y_)    
    
    return rw

In [0]:
class MLP(tf.keras.Model) :
  
  def __init__(self,input_dim):
    
    super(MLP,self).__init__()
    
    self.h1 = tf.keras.Sequential([tf.keras.layers.Dense(input_shape=(input_dim,),units=200,activation=tf.nn.relu),])
    
    self.h2 = tf.keras.Sequential([tf.keras.layers.Dense(units=300,activation=tf.nn.relu),])
    
    self.h3 = tf.keras.Sequential([tf.keras.layers.Dense(units = 500,activation=tf.nn.relu),])
    
    self.out = tf.keras.Sequential([ tf.keras.layers.Dense(units=1),])
    
  
    self.optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    self.global_step = tf.Variable(0)
    
  

  def call(self, x):
    
    x = self.h1(x)
    x = self.h2(x)
    x = self.h3(x)
    output = self.out(x)
    
    return output
    
    
  def loss(self, y,y_):
    print(y.shape, y_.shape)
    mloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=y_))
    return mloss
  
  
  def gradient(self, x,y):
    with tf.GradientTape() as tape:
        y_ = self.call(x)
        mloss = self.loss(y,y_)
        return mloss, tape.gradient(mloss,self.out.trainable_variables)
      
      
    
  def train(self, X, y, epochs, accuracy):

    for ep in range(epochs):
        print("Epoch: ", ep)
        y_ = self.call(X)
        mloss, grad = self.gradient(X,y)
        self.optimizer.apply_gradients(zip(grad, self.out.trainable_variables),global_step=self.global_step)

        print("Loss: ",mloss.numpy(), "######### "," Accuracy: ", accuracy(y,y_).numpy())

    return self.out    
 

  def predict(self):
    return self.call

In [0]:
class MLP_MMD(tf.keras.Model) :
  
  def __init__(self,input_dim):
    
    super(MLP_MMD,self).__init__()
    
    self.h1 = tf.keras.Sequential([tf.keras.layers.Dense(input_shape=(input_dim,),units=200,activation=tf.nn.relu),])
    
    self.h2 = tf.keras.Sequential([tf.keras.layers.Dense(input_shape = (200,), units=300,activation=tf.nn.relu),])
    
    self.h3 = tf.keras.Sequential([tf.keras.layers.Dense(input_shape= (300,), units = 500,activation=tf.nn.relu),])
    
    self.out = tf.keras.Sequential([ tf.keras.layers.Dense(input_shape=(500,),units=1),])
    
  
    self.optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    self.global_step = tf.Variable(0)
    
    self.custom_loss = Custom_Loss()



  

  def call(self, x):
    
    x = self.h1(x)
    x = self.h2(x)
    x = self.h3(x)
    output = self.out(x)
    
    return output
    
    
  def loss(self, y,y_ , z,z_, alpha):
    mloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=y,logits=y_))
    mmd_loss = self.custom_loss.mmd_loss(z,z_,alpha)
    loss = mloss + mmd_loss
    
    return loss
  
  
  def gradient(self, x,y, z,z_, alpha):
    with tf.GradientTape() as tape:
        y_ = self.call(x)
        loss = self.loss(y,y_, z,z_, alpha)
        return loss, tape.gradient(loss,self.out.trainable_variables)
      
      
    
  def train(self,dataset,epochs,accuracy,alpha, train_using_label='y'):

    for ep in range(epochs):
        print("Epoch: ", ep)
        
        sex_male_data,sex_male_label , sex_female_data,sex_female_label = filter_dataframe(dataset,'A',train_using_label)
        
        sex_male_data,sex_male_label , sex_female_data,sex_female_label = convert_to_tensors([sex_male_data.to_numpy(),sex_male_label.to_numpy(),sex_female_data.to_numpy(),sex_female_label.to_numpy()])
        
        X = tf.concat([sex_male_data,sex_female_data],axis=0)
        y = tf.concat([sex_male_label,sex_female_label],axis=0)
        
        y = tf.reshape(y,(y.shape[0],1))
        
        
        
        h1 = self.h1(sex_male_data)
        h2= self.h2(h1)
        z_male  = self.h3(h2)
        
        h1 = self.h1(sex_female_data)
        h2= self.h2(h1)
        z_female  = self.h3(h2)
    

        y_male = self.out(z_male)
        y_female = self.out(z_female)
        
        
        
        loss, grad = self.gradient(X,y,z_male,z_female, alpha)
        
        self.optimizer.apply_gradients(zip(grad, self.out.trainable_variables),global_step=self.global_step)
        
        if ep%10==0:
          print("Loss: ",loss.numpy(), "######### "," Male Accuracy : ", accuracy(sex_male_label,y_male).numpy(), " Female Accuracy : ", accuracy(sex_female_label,y_female).numpy())

    return self.out    
 

  def predict(self):
    return self.call

In [24]:
classifier = MLP(n)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
epochs = 1000

classifier.train(X_train,y_train, epochs, accuracy)

In [26]:
print("Total test Accuracy: ", accuracy(y_test,classifier.predict()(X_test)).numpy() )

print("Delta DP: ", delta_dp(classifier.predict(), dataset_test_frame,'sex_Female','y' )) 

print('Reweighted Accuracy: {}'.format(RW_accuracy_v2(classifier.predict(), dataset_test_frame,'A','y')))

Total test Accuracy:  0.7637592
Female Accuracy: 0.8175638318061829
Male Accuracy: 0.6390597820281982
Delta DP:  37.122173
Reweighted Accuracy: 0.7283117771148682


### For A:

In [27]:
X_train_uncr2 = X_train_frame.drop(['sex_Male', 'sex_Female'], axis = 1).to_numpy()
X_test_uncr2 = X_test_frame.drop([],axis = 1).to_numpy()

X_test_uncr2 = X_test_frame.drop(['sex_Male', 'sex_Female'], axis = 1).to_numpy()

dataset_test_frame_uncr2 = dataset_test_frame.drop(['sex_Male', 'sex_Female'], axis = 1) 

X_train_uncr2 = tf.convert_to_tensor(X_train_uncr2, dtype=tf.float32)
X_test_uncr2 = tf.convert_to_tensor(X_test_uncr2, dtype= tf.float32)


n3= X_train_uncr2.shape[1]


111


In [28]:
classifier3 = MLP(n3)


(32560, 1)


In [0]:
epochs = 1000
classifier3.train(X_train_uncr2[1:],A_train,epochs,accuracy)

In [0]:
print("Test Accuracy: ", accuracy(A_test,classifier3.predict()(X_test_uncr2[1:])).numpy() )

print("Delta DP: ", delta_dp(classifier3.predict(), dataset_test_frame_uncr2,'A','A' )) 

print('Reweighted Accuracy: {}'.format(RW_accuracy_v2(classifier3.predict(), dataset_test_frame_uncr2,'A','A')))

## Q2. Now, let’s try a neural network solution. Train a neural network with at least one hidden layer as your binary classifier. Use a cross-entropy loss, and include an MMD regularizer on the final hidden layer of the network. You can find Pytorch code for MMD in mmd.py in the assignment folder. The regularizer should measure the MMD between the internal repre- sentation of the network for each group (A = 0 or 1), multiplied by a coefficient hyperparameter α. Start with α = 0.1. Note, this is not really pre-processing, since we’re learning the solution end-to-end. As in the previous question, learn to predict Y and A with this model, and report the same results. Which method was better?

In [0]:
classifier_mmd = MLP_MMD(n)

In [0]:
epochs = 1000

classifier_mmd.train(dataset_train_frame, epochs, accuracy, 0.1)

In [0]:
print("Total test Accuracy: ", accuracy(y_test,classifier_mmd.predict()(X_test)).numpy() )

print("Delta DP: ", delta_dp(classifier_mmd.predict(), dataset_test_frame,'sex_Female','y' )) 

print('Reweighted Accuracy: {}'.format(RW_accuracy_v2(classifier_mmd.predict(), dataset_test_frame,'A','y')))

###  Training Using A:

In [14]:
classifier_mmd2 = MLP_MMD(n)

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
epochs = 1000

classifier_mmd2.train(dataset_train_frame, epochs, accuracy, 0.1, train_using_label='A') 

In [16]:
print("Total test Accuracy: ", accuracy(y_test,classifier_mmd2.predict()(X_test)).numpy() )

print("Delta DP: ", delta_dp(classifier_mmd2.predict(), dataset_test_frame,'sex_Female','y' )) 

print('Reweighted Accuracy: {}'.format(RW_accuracy_v2(classifier_mmd2.predict(), dataset_test_frame,'A','y')))

Total test Accuracy:  0.7637592
Female Accuracy: 0.8908753395080566
Male Accuracy: 0.6999998092651367
Delta DP:  3.1202517
Reweighted Accuracy: 0.7954375743865967


## Q 2.3 Hyperparameters can take a number of useful values. Try to find the useful range of the hyperparameter α. Report this range. Plot the values of accuracy and ∆ DP against various α in this range.

In [0]:
alpha_list = [0.01,1,10,100]
epochs = 1000
MMD_classifiers = [MLP_MMD(n),MLP_MMD(n),MLP_MMD(n),MLP_MMD(n)]

for i, alpha in enumerate(alpha_list):
  MMD_classifiers[i].train(dataset_train_frame, epochs, accuracy, alpha, train_using_label='y')

In [0]:
for mclassifier in MMD_classifiers:
  

  print("Test Accuracy: ", accuracy(A_test,mclassifier.predict()(X_test)).numpy() )

  print("Delta DP: ", delta_dp(mclassifier.predict(), dataset_test_frame,'A','y' )) 

  print('Reweighted Accuracy: {}'.format(RW_accuracy_v2(mclassifier.predict(), dataset_test_frame,'A','y')))

## Q 2.4  We compared two methods of removing sensitive information. What do you think is the best way to compare various methods for this task?


I think the best method is the second one using the MMD loss, because it is trying to  equalize the performance between the two sensitive attributes ( Male and Female), but there is something that observed during the training, as the accuracy for the male and female keeps fluctuating, as at the beginning the network get 100% accuracy for Male and then after some epochs it reflects its behavior to get full score for female, I think an explanation for this that it is a way for the network trying to find a point from which can find the equilibrium between the latent representation of the two genders ( sensitive attributes). 

## Q 2.5 Can you think of any other ways you might remove information about A from a representation?

In [0]:
Using VFAE network architecture.